# Confirmed Cases and Deaths Code

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
# importing fips data. and rearranging the columns
fips = pd.read_excel("~/Documents/ra/HPC/HPC_datahub/Raw data/fips.xlsx")
fips = fips.iloc[:, [1, 3, 2, 4, 5]]

# Convert the 'integers' to integers in fips file.
for i in range(fips.shape[0]):
    if fips.iloc[i, 4] != 'unknown':
        fips.iloc[i, 4] = int(fips.iloc[i, 4])
        fips.iloc[i, 2] = int(fips.iloc[i, 2])

fips = fips.drop(fips[fips.fips == 0].index)
fips = fips.drop(fips[fips.fips == "unknown"].index)
fips = fips.reset_index()
fips = fips.iloc[0:3142, 1:]
fips


,stfips,stname,ctyfips,ctyname,fips
0,1,Alabama,1,Autauga County,1001
1,1,Alabama,3,Baldwin County,1003
2,1,Alabama,5,Barbour County,1005
3,1,Alabama,7,Bibb County,1007
4,1,Alabama,9,Blount County,1009
...,...,...,...,...,...
3137,56,Wyoming,37,Sweetwater County,56037
3138,56,Wyoming,39,Teton County,56039
3139,56,Wyoming,41,Uinta County,56041
3140,56,Wyoming,43,Washakie County,56043


In [3]:
cases = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv")
cases.drop(cases.columns[[0,1,2, 3, 5, 6, 7, 8, 9, 10]], axis=1, inplace=True)

cases.rename(columns={"FIPS":"fips"}, inplace=True)

for col in cases.columns[1:]:
    cases.rename(columns={col: "confirmed_cases_" + datetime.strptime(col, "%m/%d/%y").strftime('%Y%m%d')}, inplace=True)

cases = pd.merge(fips, cases, on="fips", how="left")
#cases

In [4]:
deaths = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv")
deaths.drop(deaths.columns[[0,1,2, 3, 5, 6, 7, 8, 9, 10, 11]], axis=1, inplace=True)

deaths.rename(columns={"FIPS":"fips"}, inplace=True)

for col in deaths.columns[1:]:
    deaths.rename(columns={col: "deaths_" + datetime.strptime(col, "%m/%d/%y").strftime('%Y%m%d')}, inplace=True)

deaths = pd.merge(fips, deaths, on="fips", how="left")
#deaths

In [5]:
deaths = deaths.iloc[:, 5:]
final = pd.concat([cases, deaths], axis=1)
#final

In [6]:
# rename 'fips' column to 'scfips' to clarify it's state-county FIPS
final.rename(columns = {'fips': 'scfips'}, inplace = True)
final

,stfips,stname,ctyfips,ctyname,scfips,confirmed_cases_20200122,confirmed_cases_20200123,confirmed_cases_20200124,confirmed_cases_20200125,confirmed_cases_20200126,...,deaths_20220115,deaths_20220116,deaths_20220117,deaths_20220118,deaths_20220119,deaths_20220120,deaths_20220121,deaths_20220122,deaths_20220123,deaths_20220124
0,1,Alabama,1,Autauga County,1001,0,0,0,0,0,...,162,163,163,163,163,163,163,163,163,163
1,1,Alabama,3,Baldwin County,1003,0,0,0,0,0,...,603,605,605,605,605,607,608,608,608,608
2,1,Alabama,5,Barbour County,1005,0,0,0,0,0,...,83,83,83,83,83,83,83,83,83,83
3,1,Alabama,7,Bibb County,1007,0,0,0,0,0,...,95,95,95,95,95,95,95,95,95,95
4,1,Alabama,9,Blount County,1009,0,0,0,0,0,...,202,203,203,203,203,204,204,204,204,204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,56,Wyoming,37,Sweetwater County,56037,0,0,0,0,0,...,112,112,112,114,114,114,114,114,114,114
3138,56,Wyoming,39,Teton County,56039,0,0,0,0,0,...,14,14,14,14,14,14,14,14,14,14
3139,56,Wyoming,41,Uinta County,56041,0,0,0,0,0,...,34,34,34,34,34,34,34,34,34,34
3140,56,Wyoming,43,Washakie County,56043,0,0,0,0,0,...,41,41,41,41,41,41,41,41,41,41


In [7]:
final.to_csv("../Pandemic/casesAndDeaths.csv", index = False)

# This is the end of the main code. 
# The code below is from earlier method.

In [5]:
cols = []
for col in cases.columns[1:]:
    cols.append("confirmed_cases_" + datetime.strptime(col, "%m/%d/%y").strftime('%Y%m%d'))

In [24]:
for col in cols:
    fips[col] = 0


In [25]:
for i in range(cases.shape[0]):
    for col in cols:
        fips.loc[(fips["fips"] == cases.iloc[i, 0]), col] = cases.iloc[i, cols.index(col) + 1]



In [26]:
fips = fips.drop(fips[fips.fips == 0].index)
fips = fips.drop(fips[fips.fips == "unknown"].index)
fips = fips.iloc[0:3142]
fips

,stfips,stname,ctyfips,ctyname,fips,confirmed_cases_20200122,confirmed_cases_20200123,confirmed_cases_20200124,confirmed_cases_20200125,confirmed_cases_20200126,...,confirmed_cases_20200427,confirmed_cases_20200428,confirmed_cases_20200429,confirmed_cases_20200430,confirmed_cases_20200501,confirmed_cases_20200502,confirmed_cases_20200503,confirmed_cases_20200504,confirmed_cases_20200505,confirmed_cases_20200506
1,1,Alabama,1,Autauga County,1001,0,0,0,0,0,...,39,40,43,44,42,45,48,53,53,58
2,1,Alabama,3,Baldwin County,1003,0,0,0,0,0,...,168,171,174,174,175,181,187,188,189,196
3,1,Alabama,5,Barbour County,1005,0,0,0,0,0,...,35,37,37,39,42,43,45,45,47,47
4,1,Alabama,7,Bibb County,1007,0,0,0,0,0,...,42,42,42,42,42,42,43,42,43,43
5,1,Alabama,9,Blount County,1009,0,0,0,0,0,...,34,34,36,37,39,40,40,40,40,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3238,56,Wyoming,37,Sweetwater County,56037,0,0,0,0,0,...,16,16,17,18,18,18,18,18,18,19
3239,56,Wyoming,39,Teton County,56039,0,0,0,0,0,...,95,96,96,96,96,97,98,98,98,98
3240,56,Wyoming,41,Uinta County,56041,0,0,0,0,0,...,7,8,8,8,8,8,8,8,8,8
3241,56,Wyoming,43,Washakie County,56043,0,0,0,0,0,...,8,8,8,8,8,8,8,8,8,8


In [27]:
fips.to_csv("Confirmed_updated_05_07.csv")

# Deaths code

In [28]:
# importing fips data. and rearranging the columns
fips = pd.read_excel("fips.xlsx")
fips = fips.iloc[:, [1, 3, 2, 4, 5]]

# Convert the 'integers' to integers in fips file.
for i in range(fips.shape[0]):
    if fips.iloc[i, 4] != 'unknown':
        fips.iloc[i, 4] = int(fips.iloc[i, 4])
        fips.iloc[i, 2] = int(fips.iloc[i, 2])

In [29]:
fips = fips.drop(fips[fips.fips == 0].index)
fips = fips.drop(fips[fips.fips == "unknown"].index)
fips
fips.head()

,stfips,stname,ctyfips,ctyname,fips
1,1,Alabama,1,Autauga County,1001
2,1,Alabama,3,Baldwin County,1003
3,1,Alabama,5,Barbour County,1005
4,1,Alabama,7,Bibb County,1007
5,1,Alabama,9,Blount County,1009


In [30]:
cases = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv")
cases.drop(cases.columns[[0,1,2, 3, 5, 6, 7, 8, 9, 10, 11]], axis=1, inplace=True)

cases.head()

,FIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20
0,60.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,66.0,0,0,0,0,0,0,0,0,0,...,5,5,5,5,5,5,5,5,5,5
2,69.0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2
3,72.0,0,0,0,0,0,0,0,0,0,...,84,86,86,92,94,95,97,97,99,99
4,78.0,0,0,0,0,0,0,0,0,0,...,4,4,4,4,4,4,4,4,4,4


In [31]:
cols = []
for col in cases.columns[1:]:
    cols.append("death" + datetime.strptime(col, "%m/%d/%y").strftime('%Y%m%d'))

In [32]:
for col in cols:
    fips[col] = 0

In [33]:
for i in range(cases.shape[0]):
    for col in cols:
        fips.loc[(fips["fips"] == cases.iloc[i, 0]), col] = cases.iloc[i, cols.index(col) + 1]



In [50]:
fips = fips.drop(fips[fips.fips == 0].index)
fips = fips.drop(fips[fips.fips == "unknown"].index)
fips = fips.iloc[0:3142]
fips

,stfips,stname,ctyfips,ctyname,fips,death20200122,death20200123,death20200124,death20200125,death20200126,...,death20200423,death20200424,death20200425,death20200426,death20200427,death20200428,death20200429,death20200430,death20200501,death20200502
1,1,Alabama,1,Autauga County,1001,0,0,0,0,0,...,2,2,2,2,3,4,4,4,3,3
2,1,Alabama,3,Baldwin County,1003,0,0,0,0,0,...,3,3,3,3,3,3,3,3,4,4
3,1,Alabama,5,Barbour County,1005,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
4,1,Alabama,7,Bibb County,1007,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,Alabama,9,Blount County,1009,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3238,56,Wyoming,37,Sweetwater County,56037,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3239,56,Wyoming,39,Teton County,56039,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3240,56,Wyoming,41,Uinta County,56041,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3241,56,Wyoming,43,Washakie County,56043,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
fips.to_csv("deaths_updated_05_07.csv")

In [19]:
cases = pd.read_csv("Confirmed_updated_05_07.csv")
cases = cases.iloc[:, 1:]
cases

,stfips,stname,ctyfips,ctyname,fips,death20200122,death20200123,death20200124,death20200125,death20200126,...,death20200427,death20200428,death20200429,death20200430,death20200501,death20200502,death20200503,death20200504,death20200505,death20200506
0,1,Alabama,1,Autauga County,1001,0,0,0,0,0,...,3,4,4,4,3,3,3,3,3,3
1,1,Alabama,3,Baldwin County,1003,0,0,0,0,0,...,3,3,3,3,4,4,4,4,5,5
2,1,Alabama,5,Barbour County,1005,0,0,0,0,0,...,0,0,1,1,1,1,1,1,1,1
3,1,Alabama,7,Bibb County,1007,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,Alabama,9,Blount County,1009,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,72,Puerto Rico,145,Vega Baja Municipio,72145,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3216,72,Puerto Rico,147,Vieques Municipio,72147,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3217,72,Puerto Rico,149,Villalba Municipio,72149,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3218,72,Puerto Rico,151,Yabucoa Municipio,72151,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
deaths = pd.read_csv("deaths_updated_05_07.csv")
deaths = deaths.iloc[:, 6:]

In [61]:
merged = pd.concat([cases, deaths], axis=1)
merged

,stfips,stname,ctyfips,ctyname,fips,confirmed_cases_20200122,confirmed_cases_20200123,confirmed_cases_20200124,confirmed_cases_20200125,confirmed_cases_20200126,...,death20200423,death20200424,death20200425,death20200426,death20200427,death20200428,death20200429,death20200430,death20200501,death20200502
0,1,Alabama,1,Autauga County,1001,0,0,0,0,0,...,2,2,2,2,3,4,4,4,3,3
1,1,Alabama,3,Baldwin County,1003,0,0,0,0,0,...,3,3,3,3,3,3,3,3,4,4
2,1,Alabama,5,Barbour County,1005,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
3,1,Alabama,7,Bibb County,1007,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,Alabama,9,Blount County,1009,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,56,Wyoming,37,Sweetwater County,56037,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3138,56,Wyoming,39,Teton County,56039,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3139,56,Wyoming,41,Uinta County,56041,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3140,56,Wyoming,43,Washakie County,56043,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
os.chdir("..")
merged.to_csv("Confirmed cases and deaths.csv")